# Summary

## Fetching Data From an API: 

### Introduction to APIs

* **API Full Form:** Application Programming Interface.
* In today's software-dependent world, APIs are crucial for enabling applications to function.
* **Simple Definition:** An API allows two different software programs to communicate with each other.
* Think of APIs as **data pipelines** that transport information from one point (A) to another (B).

### Real-world Analogy: Railway Ticket Booking

* Consider booking a railway ticket via IRCTC, MakeMyTrip, or Yatra.com.
* These are different companies with their own databases.
* If you book a specific seat (e.g., B2, seat 15 on the Rajdhani Express for April 15th) through IRCTC, that seat should appear as booked on MakeMyTrip and Yatra.com as well.
* **Key Question:** How do these different platforms share real-time booking information despite having separate databases?
* **The Answer:** MakeMyTrip and Yatra.com use the **IRCTC API**.
* IRCTC exposes certain functionalities (like checking seat availability) through its API.
* When MakeMyTrip queries for seat availability, it calls the IRCTC API with the necessary information (train number, date, seat number).
* The IRCTC server responds with data, typically in **JSON format**, indicating whether the seat is available or not.
* Therefore, an API acts as a layer that allows you to interact with a database indirectly.

### Another Perspective: Social Media Platforms

* Platforms like Facebook have a single central database.
* They can have multiple interfaces (website, Android app, iOS app) that all interact with this single database through APIs.
* When you perform an action on the Android app (e.g., like a post), the change is reflected on the iOS app and website because they are all communicating with the same database via APIs.
* Understanding how APIs are built and used is a valuable skill in computer science.

### Today's Task: Fetching Movie Data from The Movie DB (TMDb) API

* We will be working with the TMDb API ([https://developers.themoviedb.org/3/](https://developers.themoviedb.org/3/)).
* TMDb provides various APIs for accessing movie data.
* We will specifically use the "Top Rated Movies" API endpoint.
* Hitting the provided URL will return a list of top-rated movies from their database.
* **Authentication:** To use the TMDb API, you need an **API Key**.
* **Obtaining an API Key:**
    1.  Go to the TMDb website.
    2.  Click on "Log in" or create an account.
    3.  Navigate to "Profile" or "Settings."
    4.  Look for a section related to "API" or "Developer."
    5.  You should find your API Key there.
* **Important Note:** Treat your API Key as confidential. There might be usage limits associated with it.

### Exploring the API Response

* Accessing the API endpoint (with the API Key) in a web browser or through code will return data in **JSON (JavaScript Object Notation)** format.
* JSON is a common data format for web APIs.
* **Understanding JSON:** JSON can be challenging to read directly. Tools like **JSON Viewer** (e.g., [https://jsonviewer.stack.hu/](https://jsonviewer.stack.hu/)) can help visualize the structure.
* **JSON Structure:** JSON resembles Python dictionaries. It consists of key-value pairs.
* In the TMDb API response for top-rated movies, the top-level might contain information like "page" and "results."
* The "results" key typically holds a **list** of movie dictionaries.
* Each dictionary within the "results" list represents a single movie and contains various details like:
    * `id`
    * `title`
    * `release_date`
    * `overview`
    * `vote_average`
    * `vote_count`
    * `popularity`
    * ...and more.
* The response might also include metadata like `total_pages` and `total_results`.

### Goal: Creating a Pandas DataFrame

* The objective is to extract relevant information from the JSON response and create a Pandas DataFrame.
* The desired columns for the DataFrame are:
    * `id`
    * `title`
    * `release_date`
    * `overview`
    * `popularity`
    * `vote_average`
    * `vote_count`
* The final DataFrame should contain data for all the top-rated movies across all pages. Based on the initial exploration, there are 428 total pages, with approximately 20 movies per page, resulting in around 8560 movies. The expected shape of the DataFrame will be (approximately 8560 rows, 7 columns).

### Step-by-Step Implementation in Python

1.  **Import Libraries:**
    ```python
    import pandas as pd
    import requests
    ```
    * `pandas` is used for data manipulation and creating DataFrames.
    * `requests` is used for making HTTP requests to the API.

2.  **Define the API Endpoint and API Key:**
    ```python
    api_key = "YOUR_API_KEY"  # Replace with your actual API key
    base_url = "https://api.themoviedb.org/3/movie/top_rated"
    ```

3.  **Make the Initial API Request:**
    ```python
    url = f"{base_url}?api_key={api_key}&page=1"  # Start with page 1
    response = requests.get(url)
    response.raise_for_status() # Raise an exception for bad status codes (e.g., 401 Unauthorized)
    data = response.json()
    ```
    * `requests.get(url)` sends an HTTP GET request to the specified URL.
    * `response.raise_for_status()` checks if the request was successful (status code 200).
    * `response.json()` parses the JSON content from the response into a Python dictionary.

4.  **Extract Results for the First Page:**
    ```python
    results = data['results']
    ```
    * Access the list of movie dictionaries from the 'results' key.

5.  **Create a Pandas DataFrame for the First Page:**
    ```python
    df = pd.DataFrame(results)
    ```
    * Convert the list of dictionaries directly into a Pandas DataFrame.

6.  **Select Desired Columns (Initial Attempt):**
    ```python
    df_selected = df[['id', 'title', 'release_date', 'overview', 'popularity', 'vote_average', 'vote_count']]
    print(df_selected.head())
    ```
    * Select the specific columns we want to keep.

7.  **Iterating Through All Pages:**
    ```python
    all_movies_df = pd.DataFrame() # Initialize an empty DataFrame

    for page_number in range(1, 429): # Loop through all the pages (1 to 428 inclusive)
        url = f"{base_url}?api_key={api_key}&page={page_number}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        temp_df = pd.DataFrame(data['results'])
        temp_df_selected = temp_df[['id', 'title', 'release_date', 'overview', 'popularity', 'vote_average', 'vote_count']]
        all_movies_df = pd.concat([all_movies_df, temp_df_selected], ignore_index=True)

    print(f"Shape of the final DataFrame: {all_movies_df.shape}")
    print(all_movies_df.head())
    ```
    * Initialize an empty DataFrame `all_movies_df`.
    * Loop through the page numbers from 1 to the total number of pages (428).
    * Construct the API URL for each page.
    * Make the API request and parse the JSON response.
    * Create a temporary DataFrame `temp_df` from the results of the current page.
    * Select the desired columns from the temporary DataFrame.
    * Use `pd.concat()` to append the `temp_df_selected` to the `all_movies_df`.
    * `ignore_index=True` ensures that the index of the final DataFrame is continuous.

8.  **Saving the DataFrame to a CSV File:**
    ```python
    all_movies_df.to_csv("top_rated_movies.csv", index=False)
    print("DataFrame saved to top_rated_movies.csv")
    ```
    * Use the `to_csv()` method to save the DataFrame to a CSV file.
    * `index=False` prevents writing the DataFrame index to the CSV file.

### Further Exploration: Kaggle and RapidAPI

* **Kaggle:** A platform for data science and machine learning. You can upload your created datasets to Kaggle, which can enhance your profile and provide opportunities for collaborations and competitions.
* **RapidAPI:** ([https://rapidapi.com/](https://rapidapi.com/)) A marketplace for various APIs across different categories (sports, finance, weather, etc.).
* You can explore the free APIs on RapidAPI, apply the same data fetching and processing techniques learned in this video, and create your own datasets.
* Uploading these datasets to platforms like Kaggle can establish you as a data expert.

### Conclusion

* Actively trying out the concepts learned is crucial for understanding.
* Explore the free APIs on RapidAPI and attempt to build your own datasets.
* This hands-on experience will solidify your understanding of API interactions and data manipulation.

# Details



---

## 📘  – Fetching Data From an API

---

### 🔹 What is an API?

**API stands for**: **Application Programming Interface**

#### 🧠 Simple Definition:

An API is a **set of rules and protocols** that allow different software applications to **communicate with each other**.

#### 🔍 Analogy:

Imagine APIs as **messengers** that take requests from you to another system and bring back the data or service you requested.

---

### 🔹 Why are APIs Important?

* In the **modern software-driven world**, apps and platforms **rely heavily on APIs** for real-time data and services.
* APIs act as **data pipelines** – helping one software send or receive data to/from another.

---

### 🔹 Real-World Example: Train Ticket Booking

#### Use Case:

Booking tickets for *Rajdhani Express* from *Howrah to Delhi* on April 15 via:

* IRCTC
* MakeMyTrip
* Yatra.com

#### What Happens Internally?

* Even though these are different apps/websites, they all reflect the **same seat availability**.
* Why? Because **they all communicate with the IRCTC database** using APIs.

#### 🔑 Key Point:

> IRCTC doesn’t share its database directly. It exposes only specific **API functions** (e.g., check seat availability) which third-party apps can call with the correct parameters.

---

### 🔹 How Does This Work?

1. **IRCTC exposes API endpoints**
2. Each function requires input (train number, date, class)
3. When MakeMyTrip calls this API:

   * It sends a request to IRCTC API
   * IRCTC processes the request and sends **JSON response** with data (like availability)
4. **The client app (e.g., MakeMyTrip)** reads this and displays to the user.

---

### 🔹 General API Structure

APIs are often accessed using URLs called **endpoints**.

For example:

```
https://api.example.com/get_movies
```

When you visit this URL or make a request to it, the API responds with data, usually in **JSON format**.

---

### 🔹 Understanding JSON (JavaScript Object Notation)

* A lightweight data-interchange format
* Very similar to Python dictionaries
* Composed of:

  * **Key-value pairs**
  * Can include lists and nested dictionaries

**Example:**

```json
{
  "page": 1,
  "results": [
    {
      "title": "Dilwale Dulhania Le Jayenge",
      "rating": 8.9
    },
    ...
  ],
  "total_pages": 28
}
```

---

### 🔹 API Use Case in the Video: Fetching Movie Data

#### API Source: [The Movie Database (TMDB)](https://www.themoviedb.org/)

1. **Go to**: [https://developers.themoviedb.org](https://developers.themoviedb.org)
2. **Create/Login** to an account
3. Go to: **Settings → API**
4. **Generate your personal API key**

---

### 🔹 Python: How to Use an API to Fetch Data

You can use the `requests` library in Python.

#### Example Code:

```python
import requests

url = "https://api.themoviedb.org/3/movie/top_rated?api_key=YOUR_API_KEY"
response = requests.get(url)

data = response.json()  # Convert JSON response to Python dictionary

# Extract and print movie titles
for movie in data['results']:
    print(movie['title'])
```

---

### 🔹 Tools to View JSON Data

If you want to better visualize the JSON data:

* Use online JSON viewers like [https://jsonviewer.stack.hu/](https://jsonviewer.stack.hu/)

---

### 🔹 Pagination in APIs

* The API returns limited results per request.
* To get more, you have to request the **next page** using:

  ```
  https://api.themoviedb.org/3/movie/top_rated?page=2
  ```

Use loops to iterate over multiple pages if needed.

---

### 🔹 Key Concepts Recap:

| Concept        | Description                                     |
| -------------- | ----------------------------------------------- |
| **API**        | Interface between software systems              |
| **Endpoint**   | URL to which API requests are sent              |
| **Request**    | Data or action you ask from the API             |
| **Response**   | Data returned by API, usually JSON              |
| **API Key**    | Secret token used to authenticate your requests |
| **Pagination** | Handling multiple pages of API responses        |
| **JSON**       | Data format used in API responses               |

---

## ✅ Practical Exercise

Try this yourself:

1. Register at [TMDB Developer](https://developers.themoviedb.org)
2. Get your API Key
3. Use Python to fetch top-rated movies
4. Display title, release date, and rating

---


